In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [52]:
#ucitavanje tog ann modela koji je treniran, skaler pickle, o_h_g (one hot encoder) pickle
model = load_model("model.h5")

#ucitavanje kodera i skalera
with open("onehot_encoder_geo.pkl","rb") as file:
    label_encoder_geo = pickle.load(file)

with open("label_encoder_gender.pkl","rb") as file:
    label_encoder_gender = pickle.load(file)

with open("scaler.pkl","rb") as file:
    scaler = pickle.load(file)
    


In [54]:
data_unos = {'CreditScore':600,
             "Geography":"France" ,
             'Gender':"Male", 
             'Age':40, 
             'Tenure':3, 
             'Balance':60000,
             'NumOfProducts':2,
             'HasCrCard':1, 
             'IsActiveMember':1, 
             'EstimatedSalary':50000,
        }

In [56]:
#'Geography_France', 'Geography_Germany', 'Geography_Spain'

In [58]:
#jedno vruce kodiranje osobine Geography
geo_kodiran = label_encoder_geo.transform([[data_unos["Geography"]]]).toarray()
geo_kodiran_df = pd.DataFrame(geo_kodiran,columns=label_encoder_geo.get_feature_names_out(["Geography"]))
geo_kodiran_df

C:\Users\amirb\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [60]:
df_unos = pd.DataFrame([data_unos])
df_unos

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [63]:
#Kodiranje Kategorickih Varijabli
df_unos["Gender"] = label_encoder_gender.transform(df_unos["Gender"])
df_unos

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [65]:
#sjedinjavanje jednog vruceg kodiranja
df_unos = pd.concat([df_unos.drop("Geography",axis=1),geo_kodiran_df],axis=1)
df_unos

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [67]:
#skaliranje podataka za unos
skaliran_unos = scaler.transform(df_unos)
skaliran_unos

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

## Predviđanje Odliva

In [70]:
#Predviđanje Odliva
predviđanje = model.predict(skaliran_unos)
predviđanje

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step


array([[0.04896443]], dtype=float32)

In [72]:
#vađenje vrijednosti rezultata predviđanja od varijable predviđanje
predviđanje_vjv = predviđanje[0][0]
predviđanje_vjv

0.048964426

In [74]:
if predviđanje_vjv >0.5:
    print("Korisnik ce najvjerovatnije napustiti banku")
else:
    print("Korisnik ce najvjerovatnije ostati uz banku")

Korisnik ce najvjerovatnije ostati uz banku
